In [1]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_stata('/root/pCloudDrive/Data/PSID_data/pequiv_long.dta')
# First we drop individuals from the SEO oversample
data = data[data.x11104LL == 'Main Sample    11']

In [34]:
variables = {'year' : "year",
        'x11101LL' : 'id',
        'd11101' : 'age',
        # Income variables
        "i11113" : "income",
        # Employment variables
        "e11101" : "anual_hours",
        # Number of persons in household
        "d11107" : "number_children"
        }

df = data[list(variables.keys())]

df.rename(columns=variables, inplace=True)

/root/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [35]:
# Filter by year
df = df[ (df.year >= 1978) & (df.year <= 1997)]

In [36]:
# Droping observations with missing values
df.dropna(inplace=True)

In [84]:
# Identifiying individuals with three years of full time employment

full_time = 52 * 40 # Full time hours per year

group = df.groupby('id')

pattern = [True, True, True]

treatment = []
control = []
neither = []

i = 0
for (id, group) in group:
    # if group.anual_hours.sum() >= full_time:
    data = group.sort_values('year')
    data['full_time_year'] = data.anual_hours >= full_time
    data["year_diff"] = data["year"].diff()
    matched = data[["full_time_year"]].rolling(len(pattern)).apply(lambda x: all(np.equal(x, pattern)))
    matched = matched.sum(axis = 1).astype(bool)
    occurences =  np.where(matched) 
    if len(occurences[0]) == 0:
        neither.append(id)
    else:
        # display(data[['year', 'age', 'anual_hours', 'full_time_year', "number_children"]])
        # print(id, occurences[0][-1], len(data))
        when  = occurences[0][-1]
        if when == len(data) - 1:
            control.append(id)
            continue
        hours_3, hours_4 = data.iloc[[when, when+1]].anual_hours.values
        children_3, children_4 = data.iloc[[when, when+1]].anual_hours.values
        if (hours_4 < .25 * hours_3) & ( children_4  <= children_3):
            treatment.append(id)
        else:
            control.append(id)
        # i += 1
        # if i > 10:
        #     break

In [100]:
sub_df = df[df.id.isin(treatment) | df.id.isin(control)]
sub_df["treatment"] = df.id.apply( lambda x : 1 if x in treatment else 0) 

<ipython-input-100-ef05a78a6898>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["treatment"] = df.id.apply( lambda x : 1 if x in treatment else 0)


In [102]:
sub_df.to_csv("/root/Work/UW-Courses/04 - 2022 Fall/Econ 810 Advanded Macroeconomic Theory/Part 1/PS 3/data/data.csv", index=False)